In [1]:
import sys
import os

import torch 
from torch import optim, nn
from torch.optim import lr_scheduler
from torch.nn import functional as F

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from pytorch_lightning import Trainer
from pytorch_lightning.core import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

from collections import OrderedDict

sys.path.append('../..')
from lib.schedulers import CosineAnnealingLRWithWarmup, DelayedScheduler

In [2]:
class MyModel(LightningModule):
    def __init__(self, model, hparams, num_workers=0, log_train_every_epoch=False):
        super().__init__()
        self.hparams = hparams
        self.model = model
        self.num_workers = num_workers
        self.log_train_every_epoch = log_train_every_epoch

    def forward(self, x):
        return self.model(x)
    
    @classmethod
    def _accuracy(cls, output, target):
        pred = output.argmax(dim=1, keepdim=True)
        eq = pred.eq(target.view_as(pred))
        return eq.all(dim=-1).float().mean()
    
    # learning rate warm-up
    def optimizer_step(self, current_epoch, batch_nb, optimizer, optimizer_i, second_order_closure=None):
        # warm up lr
        if 'sched_warmup_steps' in self.hparams:
            if self.trainer.global_step < self.hparams['sched_warmup_steps']:
                lr_scale = min(1., float(self.trainer.global_step + 1) / self.hparams['sched_warmup_steps'])
                for pg in optimizer.param_groups:
                    pg['lr'] = lr_scale * self.hparams['learning_rate']

        # update params
        optimizer.step()
        optimizer.zero_grad()

    def training_step(self, batch, batch_idx):
        images, target = batch
        output = self(images)
        # loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        acc = self._accuracy(output, target)
        lr = self.optimizer.param_groups[0]['lr']
        
        if self.log_train_every_epoch:
            return {'loss': loss, 'log': {'loss': loss.item(), 'acc': acc.item(), 'lr': lr}}
        else:
            return {'loss': loss, 'acc': acc.item(), 'lr': lr}

    def training_epoch_end(self, outputs):
        if self.log_train_every_epoch:
            return
        else:
            keys = outputs[0].keys()
            o_dict = {}
            for k in keys:
                # if k != 'loss':
                o_dict[k] = sum([o[k] for o in outputs]) / len(outputs)
            
        return {'log': o_dict}     
    
    def validation_step(self, batch, batch_idx):
        images, target = batch
        output = self(images)
        loss = F.nll_loss(output, target)
        acc = self._accuracy(output, target)

        return {'val_loss': loss.item(), 'val_acc': acc.item()}

    def validation_epoch_end(self, outputs):
        keys = outputs[0].keys()
        o_dict = {}
        for k in keys:
            o_dict[k] = sum([o[k] for o in outputs]) / len(outputs)
            
        return {'log': o_dict}
    
    def configure_optimizers(self):
        #self.optimizer = optim.SGD(
        #    self.parameters(),
        #    lr=self.hparams['learning_rate'],
        #    momentum=self.hparams['momentum'],
        #    weight_decay=self.hparams['weight_decay']
        #)
        
        self.optimizer = optim.Adam(
            self.parameters(),
            lr=self.hparams['learning_rate'],
            weight_decay=self.hparams['weight_decay']
        )        
        self.scheduler = DelayedScheduler(self.optimizer, self.hparams['sched_warmup_epoch'], 
            torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, 
                                                       self.hparams['epochs']-self.hparams['sched_warmup_epoch']))
        #self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, 
        #                                                      milestones=[100, 150], 
        #                                                      last_epoch=-1)
        #self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, 
        #                                                        gamma=self.hparams['lr_shed_gamma']) 
        
        return [self.optimizer], [self.scheduler]
    
    def prepare_data(self):
        self.normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                              std=[0.2023, 0.1994, 0.2010])

    def train_dataloader(self):
        train_loader = torch.utils.data.DataLoader(
            datasets.CIFAR10(root='./data', train=True, transform=transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.RandomCrop(32, 4),
                transforms.ToTensor(),
                self.normalize,
            ]), download=True),
            batch_size=self.hparams['batch_size'], shuffle=True,
            num_workers=self.num_workers, pin_memory=True)
        
        return train_loader

    def val_dataloader(self):
        val_loader = torch.utils.data.DataLoader(
            datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                self.normalize,
            ])),
            batch_size=self.hparams['batch_size'], shuffle=False,
            num_workers=self.num_workers, pin_memory=True)
        
        return val_loader

In [3]:
import sys

sys.path.append('../../../conv_attention_cifar/pytorch_resnet_cifar10/')

from resnet import resnet20

In [4]:
backbone_model = resnet20()

In [5]:
# epochs = 200
epochs = 120

In [6]:
model = MyModel(backbone_model, {
    'batch_size': 128,
    #'learning_rate': 0.1,
    'learning_rate': 0.004,
    #'momentum': 0.9,
    'weight_decay': 1e-4,  
    #'lr_shed_gamma': 0.96,
    #'lr_shed_gamma': 0.97,
    #'lr_shed_gamma': 0.975,
    #'sched_multiplier': 1.0, 
    'sched_warmup_epoch': 5, 
    'sched_warmup_steps': 5 * 392,
    'epochs': epochs
}, num_workers=12)

In [7]:
logger = TensorBoardLogger(
    os.getcwd(), 'lightning_logs', 'AdamLr004WarmupCosine120ep'
)
trainer = Trainer(logger, max_epochs=epochs, gpus=[1,], fast_dev_run=False)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [1]


In [8]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Reusing TensorBoard on port 6007 (pid 31516), started 1 day, 15:31:32 ago. (Use '!kill 31516' to kill it.)

In [9]:
trainer.fit(model)

INFO:lightning:
   | Name                    | Type        | Params
----------------------------------------------------
0  | model                   | ResNet      | 269 K 
1  | model.conv1             | Conv2d      | 432   
2  | model.bn1               | BatchNorm2d | 32    
3  | model.layer1            | Sequential  | 14 K  
4  | model.layer1.0          | BasicBlock  | 4 K   
5  | model.layer1.0.conv1    | Conv2d      | 2 K   
6  | model.layer1.0.bn1      | BatchNorm2d | 32    
7  | model.layer1.0.conv2    | Conv2d      | 2 K   
8  | model.layer1.0.bn2      | BatchNorm2d | 32    
9  | model.layer1.0.shortcut | Sequential  | 0     
10 | model.layer1.1          | BasicBlock  | 4 K   
11 | model.layer1.1.conv1    | Conv2d      | 2 K   
12 | model.layer1.1.bn1      | BatchNorm2d | 32    
13 | model.layer1.1.conv2    | Conv2d      | 2 K   
14 | model.layer1.1.bn2      | BatchNorm2d | 32    
15 | model.layer1.1.shortcut | Sequential  | 0     
16 | model.layer1.2          | BasicBlock  | 4 

/home/ruslan/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  warnings.warn(*args, **kwargs)


Files already downloaded and verified


1

In [10]:
trainer.save_checkpoint(os.path.join(trainer.checkpoint_callback.dirpath, "last.ckpt"))

In [11]:
#!rm -r lightning_logs

In [12]:
# trainer.test()

In [13]:
# from sklearn import metrics
# metrics.cohen_kappa_score(y, X_p, weights='quadratic')

In [14]:
'''
@dataclass
class KappaScore(ConfusionMatrix):
    "Computes the rate of agreement (Cohens Kappa)."
    weights:Optional[str]=None      # None, `linear`, or `quadratic`

    def on_epoch_end(self, last_metrics, **kwargs):
        sum0 = self.cm.sum(dim=0)
        sum1 = self.cm.sum(dim=1)
        expected = torch.einsum('i,j->ij', (sum0, sum1)) / sum0.sum()
        if self.weights is None:
            w = torch.ones((self.n_classes, self.n_classes))
            w[self.x, self.x] = 0
        elif self.weights == "linear" or self.weights == "quadratic":
            w = torch.zeros((self.n_classes, self.n_classes))
            w += torch.arange(self.n_classes, dtype=torch.float)
            w = torch.abs(w - torch.t(w)) if self.weights == "linear" else (w - torch.t(w)) ** 2
        else: raise ValueError('Unknown weights. Expected None, "linear", or "quadratic".')
        k = torch.sum(w * self.cm) / torch.sum(w * expected)
        return add_metrics(last_metrics, 1-k)
''';